<a href="https://colab.research.google.com/github/asyildirimdan/Student-Nurse-Chatbot/blob/main/%C3%96%C4%9Frenci_Hem%C5%9Fire_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================================
# PEDIATRI HEMŞİRELİĞİ RAG CHATBOT
# ============================================================================

# ADIM 1: Kütüphaneleri Yükle
# ============================================================================

import sys
!{sys.executable} -m pip install -q --upgrade google-generativeai langchain langchain-community chromadb sentence-transformers gradio

print("✅ Tüm kütüphaneler yüklendi!")


✅ Tüm kütüphaneler yüklendi!


In [2]:

# ============================================================================
# ADIM 2: Gerekli Import'lar
# ============================================================================

import os
import re
from typing import List, Dict, Tuple
import google.generativeai as genai
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import gradio as gr

print("✅ Import'lar tamamlandı!")

✅ Import'lar tamamlandı!


In [3]:

# ============================================================================
# ADIM 3: API Key Ayarla
# ============================================================================

# Colab Secrets kullanımı (Önerilen - GitHub için güvenli)
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    print("✅ API Key Colab Secrets'dan yüklendi")
except:
    # Manuel giriş - Kod çalışırken API key'inizi girin
    print("⚠️ Colab Secrets bulunamadı. Manuel giriş yapılacak.")
    GOOGLE_API_KEY = input("🔑 Google API Key'inizi girin: ")
    print("✅ API Key manuel olarak girildi")

genai.configure(api_key=GOOGLE_API_KEY)
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

✅ API Key Colab Secrets'dan yüklendi


In [4]:
# ============================================================================
# ADIM 4: Veri Dosyasını Oluştur
# ============================================================================

veri_dosyasi_icerigi = """
PEDIATRI HEMŞIRELIĞI KLINIK KILAVUZU

=== 1. YENİDOĞAN VİTAL BULGULARI ===

Yenidoğan Vital Bulgular (0-28 gün):
- Kalp Hızı: 120-160 atım/dakika (normal sınırlar: 100-180)
- Solunum Sayısı: 30-60 solunum/dakika
- Vücut Sıcaklığı: 36.5-37.5°C (aksiller)
- Kan Basıncı: 60-80/40-50 mmHg

Bebek Vital Bulgular (1-12 ay):
- Kalp Hızı: 100-150 atım/dakika
- Solunum Sayısı: 25-40 solunum/dakika
- Vücut Sıcaklığı: 36.5-37.5°C
- Kan Basıncı: 80-100/55-65 mmHg

Erken Çocukluk (1-3 yaş):
- Kalp Hızı: 90-140 atım/dakika
- Solunum Sayısı: 20-30 solunum/dakika
- Vücut Sıcaklığı: 36.5-37.2°C
- Kan Basıncı: 90-105/55-70 mmHg

Okul Öncesi (3-5 yaş):
- Kalp Hızı: 80-120 atım/dakika
- Solunum Sayısı: 20-25 solunum/dakika
- Vücut Sıcaklığı: 36.5-37°C
- Kan Basıncı: 95-110/60-75 mmHg

=== 2. İLAÇ DOZAJLARI ===

Parasetamol (Ateş Düşürücü):
- Doz: 10-15 mg/kg/doz
- Uygulama: Her 4-6 saatte bir
- Maksimum Günlük Doz: 60 mg/kg/gün veya 4 gr/gün
- Örnek: 10 kg bebek için 100-150 mg/doz
- Yol: Oral (şurup, tablet), rektal (supozituar)

İbuprofen:
- Doz: 5-10 mg/kg/doz
- Uygulama: Her 6-8 saatte bir
- Maksimum Günlük Doz: 40 mg/kg/gün
- 6 ay altı bebeklerde kullanılmaz
- Yemeklerle birlikte verilmeli

Antibiyotik - Amoksisilin:
- Doz: 25-50 mg/kg/gün, günde 2-3 doza bölünür
- Hafif enfeksiyonlar: 25 mg/kg/gün
- Orta-ağır enfeksiyonlar: 50 mg/kg/gün
- Tedavi süresi: 7-10 gün (hekimin önerisine göre)

=== 3. BESLENME REHBERİ ===

Anne Sütü (0-6 ay):
- İlk 6 ay sadece anne sütü önerilir
- Talep üzerine emzirme (2-3 saatte bir)
- Gece emzirmeleri devam etmeli
- Günlük 8-12 kez emzirme normal
- Anne sütü saklama kuralları:
  * Oda sıcaklığında (25°C): 4 saat
  * Buzdolabında (4°C): 4 gün
  * Dondurucuda (-18°C): 6 ay

Ek Gıdaya Başlama (6. ay):
- İlk gıdalar: Pirinç unu, patates püresi, havuç püresi
- Her yeni gıda 3 gün aralıkla denenmeli
- Alerjik reaksiyonlar izlenmeli
- Ballı gıdalar 1 yaşından sonra (botulizm riski)

Formül Mama:
- 0-6 ay: Birinci basamak formül
- 6-12 ay: Devam formülü
- Hazırlama: 30 ml su + 1 ölçek mama tozu
- Hazırlanan mama 1 saat içinde tüketilmeli

=== 4. GELİŞİMSEL DEĞERLENDİRME ===

0-3 Ay Motor Gelişim:
- Baş kontrolü gelişmeye başlar
- Ellerini açıp kapatır
- Yüzükoyun yatarken başını kaldırır

3-6 Ay Motor Gelişim:
- Baş kontrolü tam sağlanır
- Destekli oturma başlar
- Tuttuğu objeleri ağzına götürür

6-12 Ay Motor Gelişim:
- Desteksiz oturur (6-8 ay)
- Emekler (8-10 ay)
- Tutunarak ayağa kalkar (10-12 ay)
- İlk adımlar (12 ay civarı)

Dil Gelişimi:
- 0-3 ay: Gülümser, sesler çıkarır (gugulama)
- 3-6 ay: Yüksek sesle güler, bağırır
- 6-9 ay: Ba-ba, ma-ma gibi heceler
- 9-12 ay: İlk anlamlı kelimeler (anne, baba)
- 12-18 ay: 10-15 kelime söyler

=== 5. AŞI TAKVİMİ (Türkiye) ===

Doğumda:
- Hepatit B (1. doz)
- BCG (Verem aşısı)

2. Ay:
- KPA (Karma Aşı: Difteri, Boğmakurdu, Tetanos, Boğmaca)
- Hepatit B (2. doz)
- İPA (İnaktif Polio Aşısı)
- Pnömokok (1. doz)

4. Ay:
- KPA (2. doz)
- İPA (2. doz)
- Pnömokok (2. doz)

6. Ay:
- KPA (3. doz)
- İPA (3. doz)
- Hepatit B (3. doz)

12. Ay:
- KPA (Rapel - 4. doz)
- Pnömokok (Rapel)
- Suçiçeği
- KKK (Kızamık, Kızamıkçık, Kabakulak)

Aşı Sonrası Bakım:
- Hafif ateş (38°C altı) ve huzursuzluk normal
- Aşı yerinde kızarıklık, şişlik olabilir
- Ateş 38.5°C üzerindeyse parasetamol verilebilir
- Soğuk kompres uygulanabilir
- Bol sıvı içirilmeli

=== 6. ATEŞ YÖNETİMİ ===

Ateş Dereceleri:
- Normal: 36.5-37.5°C
- Düşük ateş: 37.5-38°C
- Orta ateş: 38-39°C
- Yüksek ateş: 39-40°C
- Çok yüksek ateş: 40°C üzeri

Ateş Ölçüm Yöntemleri:
- Aksiller (koltuk altı): En güvenli
- Rektal: En doğru, bebeklerde tercih edilir
- Oral: 4 yaş üstü
- Timpanit (kulak): Hızlı

Ateş Yönetimi:
1. Vücut ısısını doğru ölç
2. Hafif ve ince giydirin
3. Oda sıcaklığını 20-22°C tutun
4. Bol sıvı vermeye teşvik et
5. Antipiretik ilaç (parasetamol 10-15 mg/kg)
6. Ilık uygulama (28-32°C su ile)
   NOT: BUZLU SU ASLA KULLANILMAZ!

Acil Durum Kriterleri (Hemen hastaneye):
- 3 ay altı bebekte 38°C üzeri ateş
- 40°C üzeri ateş
- Ateşli nöbet
- Bilinç bulanıklığı
- Solunum güçlüğü
- 3 günden uzun süren ateş

=== 7. PEDİATRİK RESÜSITASYON ===

Temel Yaşam Desteği (BLS):
1. Güvenliği Kontrol Et
2. Tepkiyi Değerlendir (bilinci kontrol et)
3. Yardım Çağır (112)
4. Havayolunu Aç (baş-çene manevra)
5. Solunumu Kontrol Et (bak-dinle-hisset)

Göğüs Kompresyonları:

Bebek (0-12 ay):
- Teknik: İki parmak tekniği
- Konum: Meme başları hizasının 1 cm altı
- Derinlik: 4 cm (göğüs çapının 1/3'ü)
- Hız: 100-120 kompresyon/dakika

Çocuk (1-8 yaş):
- Teknik: Tek el veya iki el
- Derinlik: 5 cm
- Hız: 100-120 kompresyon/dakika

Kompresyon-Ventilasyon Oranı:
- Tek kurtarıcı: 30 kompresyon : 2 solunum
- İki kurtarıcı: 15 kompresyon : 2 solunum

Nefes Verme:
- Bebekler: Ağız-burun tekniği
- Çocuklar: Ağız-ağız tekniği
- Her solunum 1 saniye sürmeli
- Göğsün hafifçe kalkması yeterli

=== 8. ENFEKSİYON KONTROLÜ ===

El Hijyeni - 6 Adım:
1. Avuç içi avuç içine
2. Avuç içi karşı el sırtına
3. Parmak aralarını ovuştur
4. Parmak sırtlarını sürt
5. Başparmakları döndür
6. Parmak uçlarını ovuştur

El Yıkama Süreleri:
- Sabun ve su: 40-60 saniye
- Alkol bazlı antiseptik: 20-30 saniye

El Hijyeni Zamanları:
1. Hasta ile temas öncesi
2. Aseptik işlem öncesi
3. Vücut sıvısına maruziyet sonrası
4. Hasta ile temas sonrası
5. Hasta çevresiyle temas sonrası

İzolasyon Türleri:
- Standart Önlemler: Tüm hastalar için
- Temas İzolasyonu: MRSA, VRE
- Damlacık İzolasyonu: Grip, menenjit
- Havayolu İzolasyonu: Tüberküloz, kızamık

=== 9. AĞRI DEĞERLENDİRMESİ ===

FLACC Ağrı Ölçeği (2 ay - 7 yaş):
- Face (Yüz): 0-2
- Legs (Bacaklar): 0-2
- Activity (Aktivite): 0-2
- Cry (Ağlama): 0-2
- Consolability (Teselli edilebilme): 0-2
Toplam: 0-10 puan

Ağrı Yönetimi:
- Farmakolojik: Analjesikler (parasetamol, ibuprofen)
- Non-farmakolojik: Emzirme, kanguru bakımı, müzik, masaj

=== 10. DEHİDRATASYON ===

Dehidratasyon Dereceleri ve Belirtileri:

Hafif (%5):
- Kuru ağız
- Az idrara çıkma
- Huzursuzluk

Orta (%10):
- Çökük gözler
- Çökük fontanel (bebeklerde)
- Deri turgor kaybı
- Azalmış gözyaşı
- Letarji

Ağır (%15):
- Ciddi letarji
- Soğuk-nemli deri
- Zayıf nabız
- Hipotansiyon
- ACİL MÜDAHALE GEREKİR

Oral Rehidratasyon:
- ORS (Oral Rehidratasyon Solüsyonu) tercih edilir
- İlk 4 saat: 50-100 ml/kg
- Sık, küçük miktarlarda verilmeli
- Her ishal/kusma sonrası 10 ml/kg ek sıvı

Bu dosya, pediatri alanına yeni adım atan öğrenci hemşireler için kapsamlı bir rehberdir.
Amaç, çocuk sağlığı hemşireliğinde bilgi, gözlem ve empatiyi birleştirerek güvenli ve etkili bakım sağlamaktır.


1. PEDİATRİK VİTAL BULGULAR
Yaşa göre vital bulgular pediatride büyük farklılık gösterir.
Küçük çocukların kalp atımı ve solunumu hızlı, tansiyonu düşüktür. Normal değerlerin dışına çıkan ölçümler, yaşla orantılı değerlendirilmelidir.

- Yenidoğan (0–28 gün): Nabız 100–180/dk, Solunum 30–60/dk, Ateş 36.5–37.5°C, Tansiyon 60/40 mmHg
- Bebek (1–12 ay): Nabız 100–160/dk, Solunum 30–50/dk, Ateş 36.5–37.5°C, Tansiyon 80/50 mmHg
- Küçük çocuk (1–3 yaş): Nabız 90–150/dk, Solunum 25–40/dk, Ateş 36–37.5°C, Tansiyon 90/60 mmHg
- Okul öncesi: Nabız 80–140/dk, Solunum 20–30/dk, Ateş 36–37.5°C, Tansiyon 95/60 mmHg
- Okul çağı: Nabız 70–120/dk, Solunum 18–25/dk, Ateş 36–37.5°C, Tansiyon 100/65 mmHg
- Ergen: Nabız 60–100/dk, Solunum 12–20/dk, Ateş 36–37.5°C, Tansiyon 110/70 mmHg

🩺 İpucu: Nabız ölçümü bebekte brakial, okul çağında radial arterden alınmalıdır.


2. ÇOCUKLA İLETİŞİM VE YAKLAŞIM
İletişim, pediatri hemşireliğinin temelidir. Çocuğun yaşına ve gelişim düzeyine göre iletişim biçimi değişir.

- Bebeklerde: Dokunma, ses tonu ve göz temasıyla güven oluşturulur.
- Okul öncesi: Oyunla anlatmak etkili bir yöntemdir. “Şimdi bu stetoskopla kalbini dinleyeceğim” gibi açıklamalar kullan.
- Okul çağı: Neden-sonuç ilişkisi kurabilir. Ona işlemi anlat ve kontrol hissi ver.
- Ergen: Kendi alanına ve gizliliğine saygı duy; tedavi kararlarına dahil et.

💡 Örnek: Kan alırken “Şimdi biraz batacak ama hemen geçecek, ister misin elimi tutsun annen?” demek, güveni artırır.


3. FİZİK MUAYENEDE DİKKAT EDİLECEK NOKTALAR
- Önce çocuğu gözlemle. Ağlama, nefes alma, cilt rengi ve kas tonusu ipucu verir.
- Muayene sırası: İzle → Palpe et → Perküsyon → Oskültasyon.
- Rahatsız edici işlemleri (örneğin kulak ve boğaz muayenesi) en sona bırak.
- Eller sıcak olmalı, ortam sessiz ve sıcak tutulmalıdır.
- İşlem sırasında aileden destek al, ama çocukla direkt iletişimi koru.


4. PEDİATRİK BAKIMDA ÖZEL UYGULAMALAR
- Ağırlık takibi çok önemlidir; her sabah aynı saatte, aynı tartı cihazıyla ölçülmelidir.
- Sıvı dengesi izlenmeli: Normal idrar çıkışı ≥1 ml/kg/saat.
- Cilt bakımı: Hassas ciltlerde sabun yerine pH nötr temizleyiciler kullanılmalı.
- Aşı sonrası: Hafif ateş ve kızarıklık normaldir; aileye açıklama yapılmalıdır.
- Yenidoğanda göbek bakımı steril gaz ve antiseptikle yapılır.


5. PEDİATRİK İLAÇ UYGULAMALARI
Pediatrik ilaç dozları kilogram başına hesaplanır.

💊 **Genel formül:**
Doz (mg) = Birim doz (mg/kg) × Vücut ağırlığı (kg)

🧮 **Örnek:** Parasetamol tek doz = 15 mg/kg
Eğer çocuk 12 kg ise: 15 × 12 = 180 mg tek doz verilir.
(Genellikle günde 4–6 saatte bir uygulanır, maksimum günlük doz 60 mg/kg geçmemelidir.)

Kas içi enjeksiyonlarda vastus lateralis kası (uyluğun dış yan kısmı) tercih edilir.
İğne uzunluğu çocuğun kilosuna göre ayarlanmalıdır.
Ağrıyı azaltmak için dikkat dağıtıcı yöntemler kullanılabilir (örneğin balon şişirmek).


6. AİLE İLE İLETİŞİM VE EBEVEYN ROLÜ
- Aile, pediatrik bakımın merkezindedir. Onları sürece aktif olarak dahil et.
- Aileye bakım sürecinde roller ver (örneğin beslenme, gözlem, rahatlatma).
- Ebeveyn kaygısı çocukta da strese yol açar; sakin, açıklayıcı bir dil kullan.
- Bilgi verirken tıbbi terimleri sadeleştir: “Enfeksiyon” yerine “mikrop girmiş”.


7. GELİŞİM DÖNEMLERİNE GÖRE HEMŞİRELİK YAKLAŞIMI
- **Bebek (0–1 yaş):** Güven duygusunu oluşturmak için dokunsal temas önemlidir. Düzenli beslenme, ısı kontrolü ve ağlama nedenlerini gözlemle.
- **Okul öncesi (1–5 yaş):** Oyunla iş birliği sağlanır. "Şimdi oyunumuz iğneyle cesur olma oyunu" gibi ifadeler kullanılabilir.
- **Okul çağı (6–12 yaş):** Bilgiye açtır. Her işlem öncesi ne yapılacağını anlat.
- **Ergen (13–18 yaş):** Mahremiyetine saygı göster, duygularını dinle. Onu tedavi sürecine dahil et.


8. KLİNİKTE ÖĞRENCİ HEMŞİRELERE ÖNERİLER
- İlk günlerde sadece gözlem yapmak öğrenmenin en güçlü yoludur.
- Hata yapmaktan korkma ama asla gizleme; öğrenmek cesaret ister.
- Çocuğa yaklaşmadan önce kendini tanıt. "Ben hemşire Senay, seninle ilgileneceğim."
- Hastane ortamında sabır, empati ve dikkat senin en büyük araçların.
- Küçük bir dokunuş, sıcak bir ses tonu çocuğun tüm gününü değiştirebilir.


9. ETİK VE EMPATİ
- Çocuğun mahremiyetini koru. Muayene veya işlem öncesi izin iste.
- Çocuğa “korkacak bir şey yok” deme; bunun yerine “korkman normal ama ben yanındayım” de.
- Aileden izin almadan işlem yapılmaz.
- Her çocuk saygı, güven ve şefkatle yaklaşılmayı hak eder.

🧩 Örnek: Bir çocuk enjeksiyon sonrası ağladığında "Cesurdun, ağlamak utanılacak bir şey değil" diyebilirsin.


10. PEDİATRİDE YETİŞKİNDEN FARKLI OLAN TEMEL NOKTALAR
- Çocuklar küçük yetişkinler değildir.
- Fizyolojik parametreleri, metabolizmaları, ilaç yanıtları ve duygusal tepkileri farklıdır.
- İletişim kurarken yaşa uygun dil ve yöntem seçimi esastır.
- Pediatri hemşiresi gözlem yeteneğini, sabrını ve sevgisini birlikte kullanır.


11. APGAR SKORU (YENİDOĞAN DEĞERLENDİRME)
APGAR, doğumdan sonraki 1. ve 5. dakikada yenidoğanın genel durumunu değerlendirmek için kullanılır.
Beş parametre değerlendirilir; her biri 0–2 puan arası verilir.

PARAMETRELER VE PUANLAMA:
- Kalp Atım Hızı: 0 = yok, 1 = <100/dk, 2 = >100/dk
- Solunum: 0 = yok, 1 = yavaş veya düzensiz, 2 = iyi, ağlama var
- Kas Tonusu: 0 = gevşek, 1 = az hareket, 2 = aktif hareket
- Renk: 0 = mor soluk, 1 = vücut pembe, ekstremiteler mor, 2 = tamamen pembe
- Refleks İrritabilitesi: 0 = yanıt yok, 1 = yüz buruşturma, 2 = ağlama/çekilme

Toplam puan:
7–10 = Normal, 4–6 = Orta asfiksi, 0–3 = Ağır asfiksi.


12. MİNİ SÖZLÜK
- **APGAR Skoru:** Yenidoğanın doğum sonrası yaşamsal durumunu 1. ve 5. dakikada 5 parametreyle değerlendiren sistem.
- **Fontanel:** Bebek kafatasındaki bıngıldak; ön fontanel 9–18 ay arasında kapanır.
- **Ruminasyon:** Yenidoğanın mide içeriğini tekrar ağza getirme davranışı.
- **Hipoglisemi:** Kan şekeri düşüklüğü (yeni doğanda <45 mg/dL tehlikelidir).
- **Desatürasyon:** Oksijen satürasyonunun %90’ın altına düşmesidir.
- **Seftriakson:** Geniş spektrumlu antibiyotik; yenidoğanda kalsiyumlu solüsyonlarla karıştırılmamalıdır.
- **Kateter:** İlaç veya sıvı uygulaması için kullanılan ince tüp.


SON SÖZ 💚
Pediatri hemşireliği sadece bilgi değil, aynı zamanda kalp işidir.
Bir çocuk senin ses tonunu, enerjini ve ilgini hisseder.
Her bakım anı, öğrenme ve sevgiyle yapılan küçük bir mucizedir.

"""

with open('pediatri_hemsirelik_bilgi_dosyasi.txt', 'w', encoding='utf-8') as f:
    f.write(veri_dosyasi_icerigi)

print("✅ Veri dosyası oluşturuldu!")
print(f"   Boyut: {len(veri_dosyasi_icerigi)} karakter")

✅ Veri dosyası oluşturuldu!
   Boyut: 12848 karakter


In [5]:

# ============================================================================
# ADIM 5: Veri İşleme Sınıfı
# ============================================================================

class PediatriDataProcessor:
    def __init__(self, dosya_yolu: str):
        self.dosya_yolu = dosya_yolu
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=800,
            chunk_overlap=100,
            separators=["\n\n", "\n", ". ", " ", ""]
        )

    def veri_yukle(self) -> str:
        try:
            with open(self.dosya_yolu, 'r', encoding='utf-8') as f:
                icerik = f.read()
            print(f"✅ Veri yüklendi: {len(icerik)} karakter")
            return icerik
        except Exception as e:
            print(f"❌ Hata: {str(e)}")
            return ""

    def temizle(self, metin: str) -> str:
        metin = re.sub(r'\s+', ' ', metin)
        metin = metin.strip()
        return metin

    def parcalara_bol(self, metin: str) -> List[str]:
        temiz_metin = self.temizle(metin)
        parcalar = self.text_splitter.split_text(temiz_metin)
        print(f"📦 Metin {len(parcalar)} parçaya bölündü")
        return parcalar

print("✅ Veri işleme sınıfı hazır!")

✅ Veri işleme sınıfı hazır!


In [6]:

# ============================================================================
# ADIM 6: RAG Engine Sınıfı
# ============================================================================

class PediatriRAGEngine:
    def __init__(self, api_key: str):
        genai.configure(api_key=api_key)

        # HuggingFace Embedding (Ücretsiz, Türkçe destekli)
        print("🔄 Embedding modeli yükleniyor...")
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
            model_kwargs={'device': 'cpu'}
        )

        # Gemini 1.5 Flash (gemini-pro yerine)
        self.model = genai.GenerativeModel('gemini-1.5-flash')
        self.vectordb = None
        print("✅ RAG Engine başlatıldı (Gemini 1.5 Flash + HuggingFace)")

    def vektor_db_olustur(self, parcalar: List[str]):
        print("🔄 Vektör veritabanı oluşturuluyor...")

        documents = [Document(page_content=parca) for parca in parcalar]

        self.vectordb = Chroma.from_documents(
            documents=documents,
            embedding=self.embeddings,
            collection_name="pediatri_hemsirelik"
        )

        print(f"✅ Vektör veritabanı oluşturuldu: {len(parcalar)} doküman")
        return self.vectordb

    def benzer_dokumanlar_bul(self, soru: str, k: int = 4):
        if not self.vectordb:
            raise ValueError("❌ Vektör veritabanı yüklenmemiş!")

        docs_with_scores = self.vectordb.similarity_search_with_score(soru, k=k)
        return docs_with_scores

    def cevap_uret(self, soru: str, benzer_doklar):
        context_parcalari = []
        for doc, score in benzer_doklar:
            if score < 1.5:
                context_parcalari.append(doc.page_content)

        context = "\n\n---\n\n".join(context_parcalari)

        prompt = f"""Sen bir pediatri hemşireliği eğitim asistanısın.
Hemşirelik öğrencilerine klinik bilgiler sunuyorsun.

KURALLAR:
1. Sadece verilen bilgilere dayanarak cevap ver
2. Medikal bilgileri net ve anlaşılır şekilde açıkla
3. Dozaj ve sayısal bilgilerde çok dikkatli ol
4. Emin olmadığın bilgileri ekleme
5. Türkçe ve profesyonel bir dil kullan
6. Gerekirse madde madde açıkla

KAYNAK BİLGİLER:
{context}

ÖĞRENCİ SORUSU: {soru}

CEVAP:"""

        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"❌ Cevap üretilirken hata oluştu: {str(e)}"

    def soru_cevap(self, soru: str):
        benzer_doklar = self.benzer_dokumanlar_bul(soru, k=4)
        cevap = self.cevap_uret(soru, benzer_doklar)

        kaynaklar = []
        for i, (doc, score) in enumerate(benzer_doklar[:3], 1):
            kaynaklar.append({
                'sira': i,
                'icerik': doc.page_content[:150] + "...",
                'skor': round(score, 3)
            })

        return {
            'soru': soru,
            'cevap': cevap,
            'kaynaklar': kaynaklar
        }

print("✅ RAG Engine sınıfı hazır!")

✅ RAG Engine sınıfı hazır!


In [19]:
# ============================================================================
# ADIM 7: RAG Engine Sınıfı (AKILLI ARAMA - DÜZELTİLDİ)
# ============================================================================

class PediatriRAGEngine:
    def __init__(self, api_key: str):
        genai.configure(api_key=api_key)

        print("🔄 Embedding modeli yükleniyor...")
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
            model_kwargs={'device': 'cpu'}
        )

        self.model = genai.GenerativeModel('gemini-pro')
        self.vectordb = None
        print("✅ RAG Engine başlatıldı")

    def vektor_db_olustur(self, parcalar: List[str]):
        print("🔄 Vektör veritabanı oluşturuluyor...")

        self.vectordb = Chroma.from_texts(
            texts=parcalar,
            embedding=self.embeddings,
            collection_name="pediatri_hemsirelik"
        )

        print(f"✅ Vektör veritabanı oluşturuldu: {len(parcalar)} doküman")
        return self.vectordb

    def benzer_dokumanlar_bul(self, soru: str, k: int = 6):
        if not self.vectordb:
            raise ValueError("❌ Vektör veritabanı yüklenmemiş!")

        docs_with_scores = self.vectordb.similarity_search_with_score(soru, k=k)
        return docs_with_scores

    def anahtar_kelime_skoru(self, soru: str, dokuman: str) -> int:
        """Sorudaki önemli kelimelerin dokümanda olup olmadığını kontrol et"""
        soru_lower = soru.lower()
        dokuman_lower = dokuman.lower()

        # Önemli anahtar kelimeler
        onemli_kelimeler = [
            "apgar", "fontanel", "aşı", "parasetamol", "ibuprofen",
            "nabız", "kalp", "solunum", "tansiyon", "ateş",
            "kompresyon", "resüsitasyon", "el hijyen", "dehidratasyon",
            "anne sütü", "beslenme", "gelişim", "doz"
        ]

        skor = 0

        # Önemli kelimeler varsa +10 puan
        for kelime in onemli_kelimeler:
            if kelime in soru_lower and kelime in dokuman_lower:
                skor += 10

        # Sorudaki kelimelerin kaçı dokümanda var
        soru_kelimeleri = [k for k in soru_lower.split() if len(k) > 3]
        for kelime in soru_kelimeleri:
            if kelime in dokuman_lower:
                skor += 1

        return skor

    def cevap_uret(self, soru: str, benzer_doklar):
        """Hem benzerlik hem anahtar kelime skoruna göre en iyi dokümanı seç"""

        en_iyi_doc = None
        en_iyi_skor = -1

        for doc, similarity_score in benzer_doklar:
            # Anahtar kelime skoru hesapla
            keyword_score = self.anahtar_kelime_skoru(soru, doc.page_content)

            # Toplam skor = anahtar kelime skoru - (similarity score / 10)
            # (similarity score düşükse iyi, o yüzden eksi)
            toplam_skor = keyword_score - (similarity_score / 10)

            if toplam_skor > en_iyi_skor:
                en_iyi_skor = toplam_skor
                en_iyi_doc = doc.page_content

        if not en_iyi_doc:
            return "❌ Üzgünüm, bu soruyla ilgili bilgi bulamadım."

        # Cevabı formatla
        cevap = f"**📋 Cevap:**\n\n{en_iyi_doc}"

        return cevap

    def soru_cevap(self, soru: str):
        benzer_doklar = self.benzer_dokumanlar_bul(soru, k=6)
        cevap = self.cevap_uret(soru, benzer_doklar)

        kaynaklar = []
        for i, (doc, score) in enumerate(benzer_doklar[:3], 1):
            kaynaklar.append({
                'sira': i,
                'icerik': doc.page_content[:150] + "...",
                'skor': round(score, 3)
            })

        return {
            'soru': soru,
            'cevap': cevap,
            'kaynaklar': kaynaklar
        }

print("✅ RAG Engine sınıfı hazır!")

✅ RAG Engine sınıfı hazır!


In [20]:
# ============================================================================
# ADIM 8: Sistemi Başlat (GÜNCELLENMİŞ)
# ============================================================================

print("\n" + "="*60)
print("🚀 SİSTEM BAŞLATILIYOR...")
print("="*60)

processor = PediatriDataProcessor('pediatri_hemsirelik_bilgi_dosyasi.txt')
metin = processor.veri_yukle()
parcalar = processor.parcalara_bol(metin)

# RAG engine oluştur
rag_engine = PediatriRAGEngine(GOOGLE_API_KEY)

# Vektör DB'yi kontrol et
if rag_engine.vectordb is None:
    print("🔄 Vektör veritabanı oluşturuluyor...")
    rag_engine.vektor_db_olustur(parcalar)
else:
    print("✅ Vektör veritabanı zaten mevcut!")

print("\n✅ SİSTEM HAZIR!\n")

# Test et
print("🧪 Sistem testi yapılıyor...")
try:
    test_sonuc = rag_engine.soru_cevap("Yenidoğanda normal kalp hızı kaç?")
    print(f"✅ Test başarılı! Cevap uzunluğu: {len(test_sonuc['cevap'])} karakter")
except Exception as e:
    print(f"⚠️ Test hatası: {str(e)}")


🚀 SİSTEM BAŞLATILIYOR...
✅ Veri yüklendi: 12848 karakter
📦 Metin 20 parçaya bölündü
🔄 Embedding modeli yükleniyor...
✅ RAG Engine başlatıldı
🔄 Vektör veritabanı oluşturuluyor...
🔄 Vektör veritabanı oluşturuluyor...
✅ Vektör veritabanı oluşturuldu: 20 doküman

✅ SİSTEM HAZIR!

🧪 Sistem testi yapılıyor...
✅ Test başarılı! Cevap uzunluğu: 788 karakter


In [21]:
# ============================================================================
# ADIM 9: Gradio Arayüzü - ÖRNEK SORULAR GÜNCELLENDİ
# ============================================================================

def chatbot_fonksiyonu(soru, gecmis):
    """Chatbot fonksiyonu - Hata kontrolü ile"""

    if not soru.strip():
        return gecmis, ""

    gecmis = gecmis + [[soru, None]]

    try:
        if rag_engine.vectordb is None:
            gecmis[-1][1] = "❌ Vektör veritabanı yüklenmemiş. Lütfen sistemi yeniden başlatın."
            return gecmis, ""

        sonuc = rag_engine.soru_cevap(soru)
        cevap = sonuc['cevap']

        if sonuc['kaynaklar']:
            cevap += "\n\n---\n**📚 Kaynak Bilgiler:**\n"
            for kaynak in sonuc['kaynaklar']:
                cevap += f"\n**Kaynak {kaynak['sira']}** (Benzerlik: {kaynak['skor']})\n"
                cevap += f"_{kaynak['icerik']}_\n"

        gecmis[-1][1] = cevap

    except Exception as e:
        hata_mesaji = str(e)
        if "quota" in hata_mesaji.lower():
            gecmis[-1][1] = "⚠️ API kotası doldu. Lütfen birkaç saniye bekleyin."
        elif "rate" in hata_mesaji.lower():
            gecmis[-1][1] = "⚠️ Çok hızlı istek gönderdiniz. 5 saniye bekleyin."
        else:
            gecmis[-1][1] = f"❌ Hata: {hata_mesaji[:200]}"

    return gecmis, ""

# VERİ TABANINA UYGUN ÖRNEK SORULAR (GÜNCELLENDİ)
ornek_sorular = [
    "Yenidoğanda normal nabız kaç olmalı?",
    "Bebeklerde Apgar skoru nasıl hesaplanır?",
    "Anne sütü buzdolabında ne kadar saklanabilir?",
    "2 yaşında çocukta normal solunum sayısı nedir?",
    "Parasetamol dozu nasıl hesaplanır?",
    "Pediatrik resüsitasyonda kompresyon-ventilasyon oranı nedir?",
    "Fontanel ne zaman kapanır?",
    "Dehidratasyon belirtileri nelerdir?",
    "El hijyeni 6 adımı nelerdir?",
    "Türkiye aşı takviminde 2. ayda hangi aşılar yapılır?",
    "Okul çağı çocukta normal tansiyon değerleri nelerdir?",
    "İlaç enjeksiyonunda hangi kas bölgesi tercih edilir?",
    "Ergenlerle iletişim nasıl kurulmalı?",
    "FLACC ağrı ölçeği nedir?",
    "Ateş yönetiminde buzlu su kullanılır mı?"
]

with gr.Blocks(theme=gr.themes.Soft(), title="Pediatri Hemşireliği Klinik Kılavuz") as demo:

    gr.Markdown("""
    # 👶 Pediatri Hemşireliği Klinik Kılavuz Asistanı

    Pediatrik hasta bakımı, vital bulgular, ilaç dozajları, gelişim değerlendirmesi ve acil durumlar hakkında sorularınızı sorun.

    ⚠️ **Önemli:** Bu asistan eğitim amaçlıdır. Gerçek klinik kararlar için mutlaka süpervizörünüze danışın.
    """)

    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(
                value=[],
                height=500,
                label="Sohbet Geçmişi"
            )

            with gr.Row():
                soru_input = gr.Textbox(
                    placeholder="Sorunuzu buraya yazın... (örn: 'Bebeklerde Apgar skoru nedir?')",
                    label="Soru",
                    lines=2,
                    scale=4
                )
                gonder_btn = gr.Button("📤 Gönder", variant="primary", scale=1)

            temizle_btn = gr.Button("🗑️ Sohbeti Temizle")

        with gr.Column(scale=1):
            gr.Markdown("### 💡 Örnek Sorular")
            gr.Markdown("Veri tabanımızdan örnek sorular:")

            for ornek in ornek_sorular:
                ornek_btn = gr.Button(ornek, size="sm")
                ornek_btn.click(
                    fn=lambda x=ornek: x,
                    outputs=soru_input
                )

    gr.Markdown("""
    ---
    ### 📊 Proje Bilgileri

    **Teknolojiler:** Gemini 1.5 Flash • HuggingFace Embeddings • ChromaDB • Gradio

    **Veri Kaynağı:**
    - Pediatrik vital bulgular ve değerlendirme
    - İlaç dozaj hesaplamaları
    - Gelişimsel milestone'lar
    - Aşı takvimi (Türkiye)
    - Pediatrik resüsitasyon protokolleri
    - El hijyeni ve enfeksiyon kontrolü
    - APGAR skoru ve yenidoğan bakımı

    **Akbank GenAI Bootcamp Projesi - 2025**
    """)

    gonder_btn.click(
        fn=chatbot_fonksiyonu,
        inputs=[soru_input, chatbot],
        outputs=[chatbot, soru_input]
    )

    soru_input.submit(
        fn=chatbot_fonksiyonu,
        inputs=[soru_input, chatbot],
        outputs=[chatbot, soru_input]
    )

    temizle_btn.click(
        fn=lambda: ([], ""),
        outputs=[chatbot, soru_input]
    )

print("✅ Gradio arayüzü hazırlandı!")

✅ Gradio arayüzü hazırlandı!


/tmp/ipython-input-1234266409.py:71: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


In [22]:
# ============================================================================
# ADIM 10: Arayüzü Başlat
# ============================================================================

print("\n" + "="*60)
print("🎉 GRADIO ARAYÜZÜ BAŞLATILIYOR...")
print("="*60)
print("\n📱 Arayüz linkine tıklayarak chatbot'u kullanabilirsiniz!")
print("🌐 Public link ile herkesle paylaşabilirsiniz (72 saat aktif)!\n")

demo.launch(share=True, debug=True)


🎉 GRADIO ARAYÜZÜ BAŞLATILIYOR...

📱 Arayüz linkine tıklayarak chatbot'u kullanabilirsiniz!
🌐 Public link ile herkesle paylaşabilirsiniz (72 saat aktif)!

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://916c72116991edaa7d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://916c72116991edaa7d.gradio.live
